#### __1° Passo__: Leitura do arquivo para compreensão do problema.
- Ler os arquivos e juntalos todos de uma vez.

In [ ]:
# Importando as bibliotecas necessárias
import pandas as pd # type: ignore
import numpy as np # type: ignore
import os
# pegando a pasta atual
path = os.getcwd()
file_folder = 'Arquivos/'
file_path = os.path.join(path, file_folder)
file_paths = os.listdir(file_path)
# criando um dataframe vazio
df_file = pd.DataFrame()
# iterando sobre os arquivos
for file in file_paths:
   print(file)
   df_files = pd.read_csv(file_path + file, sep = ';', encoding='latin1', header=4)
   print(f'Quantidade de linhas: {df_files.shape}')
   df_file = pd.concat([df_file, df_files])
   print(f'Quantidade de linhas total: {df_file.shape}')

df_file.head(2)

RELATORIO_REDES_GUIA_MEDICO_SP103_13032025093343.csv
Quantidade de linhas: (258432, 32)
Quantidade de linhas total: (258432, 32)
RELATORIO_REDES_GUIA_MEDICO_SP513_13032025100010.csv
Quantidade de linhas: (73185, 32)
Quantidade de linhas total: (331617, 32)
RELATORIO_REDES_GUIA_MEDICO_SP519_13032025095640.csv
Quantidade de linhas: (69867, 32)
Quantidade de linhas total: (401484, 32)
RELATORIO_REDES_GUIA_MEDICO_SP523_13032025095359.csv
Quantidade de linhas: (142732, 32)
Quantidade de linhas total: (544216, 32)


,CONTRATACAO,NU_PRODUTO,NM_PRODUTO,REDE_ATENDIMENTO,DS_REDE_ATENDIMENTO,FL_ABRANGENCIA,UF,CIDADE,TIPO_ESTABELECIMENTO,SERVICO,...,CD_CEP,DS_FONE,SITE,NM_LIVRO,PRESTADOR,TIPO_PESSOA,NM_LOCAL_ATENDIMENTO,IBGE,TIPO_PRESTADOR_SERVICO,SITUACAO_PRODUTO
0,2-Coletivo Empresarial,474363156,ADVANCE 600,SP103,ADVANCE 600,3,AC,RIO BRANCO,HOSPITAL ELETIVO,1,...,69908650.0,6832241364/,NaN,AMICO LIMITADA,AMICO LIMITADA,2,PRONTO CLINICA,1200401,1-HOSPITAL,1-ATIVO
1,2-Coletivo Empresarial,474363156,ADVANCE 600,SP103,ADVANCE 600,3,AC,RIO BRANCO,SERVICOS AUXILIARES DE DIAGNOSTICO E TERAPIA,1,...,69908650.0,6832241364/,NaN,AMICO LIMITADA,AMICO LIMITADA,2,PRONTO CLINICA,1200401,1-HOSPITAL,1-ATIVO


#### __2° Passo__: Busca por produtos específicos.
- listar os produtos que desejamos buscar no arquivo.

In [8]:
# Para facilitar o mecanismo de busca, vamos transformar a coluna NU_PRODUTO em string
df_file.NU_PRODUTO = df_file.NU_PRODUTO.astype(str)

In [9]:
# Mostrar todos os produtos únicos para teste
print(df_file.NU_PRODUTO.nunique())
print(df_file.NU_PRODUTO.unique())


15
['474363156' '474337157' '474439150' '474441151' '486576206' '486579201'
 '486580204' '486577204' '474460158' '474464151' '474462154' '474453155'
 '474449157' '474448159' '474451159']


In [10]:
# Vamos fazer uma busca por produtos específicos, lista de pesquisa
search = ['474363156', '486576206', '474462154', '474449157']
#df[df.NU_PRODUTO.isin(search)].head(2)

# fitrando o dataframe
df_filtered = df_file[df_file.NU_PRODUTO.isin(search)]
df_filtered.reset_index(drop=True, inplace=True)
print(f'Quantidade de linhas do df original: {df_file.shape}')
print(f'Quantidade de linhas filtradas: {df_filtered.shape}')
df_filtered.head(2)


Quantidade de linhas do df original: (544216, 32)
Quantidade de linhas filtradas: (141868, 32)


,CONTRATACAO,NU_PRODUTO,NM_PRODUTO,REDE_ATENDIMENTO,DS_REDE_ATENDIMENTO,FL_ABRANGENCIA,UF,CIDADE,TIPO_ESTABELECIMENTO,SERVICO,...,CD_CEP,DS_FONE,SITE,NM_LIVRO,PRESTADOR,TIPO_PESSOA,NM_LOCAL_ATENDIMENTO,IBGE,TIPO_PRESTADOR_SERVICO,SITUACAO_PRODUTO
0,2-Coletivo Empresarial,474363156,ADVANCE 600,SP103,ADVANCE 600,3,AC,RIO BRANCO,HOSPITAL ELETIVO,1,...,69908650.0,6832241364/,NaN,AMICO LIMITADA,AMICO LIMITADA,2,PRONTO CLINICA,1200401,1-HOSPITAL,1-ATIVO
1,2-Coletivo Empresarial,474363156,ADVANCE 600,SP103,ADVANCE 600,3,AC,RIO BRANCO,SERVICOS AUXILIARES DE DIAGNOSTICO E TERAPIA,1,...,69908650.0,6832241364/,NaN,AMICO LIMITADA,AMICO LIMITADA,2,PRONTO CLINICA,1200401,1-HOSPITAL,1-ATIVO


#### __3° Passo__: Excluíndo colunas indesejadas:
- 3.1:
   - `FL_ABRANGENCIA`
   - `CD_REDE_ANS`
   - `NM_COMERCIAL`
   - `PRESTADOR`
   - `NM_LOCAL_ATENDIMENTO`
   - `SITUACAO_PRODUTO`
   - __`CD_ESPECIALIDADE`__ Está em __Observação__.
- 3.2:
   - Trazendo a coluna `NM_PRESTADOR` para a direita da coluna `CIDADE`.
   - Trazendo a coluna `NM_FANTASIA` para a direita da coluna `CIDADE`.

In [253]:
# Vamos criar uma lista de colunas que não são necessárias para a análise, no total são 6 colunas.
colums_to_delete = ['FL_ABRANGENCIA', 'CD_REDE_ANS', 'NM_COMERCIAL', 'PRESTADOR', 'NM_LOCAL_ATENDIMENTO', 'SITUACAO_PRODUTO']

# mostrando a quantidade de colunas antes e depois da exclusão
print(f'Quantidade de colunas antes da exclusão: {df_filtered.shape[1]}')
# deletando as colunas
df_colums_deleted = df_filtered.copy()
df_colums_deleted.drop(columns=colums_to_delete, inplace=True)
# mostrando a quantidade de colunas após a exclusão
print(f'Quantidade de colunas após a exclusão: {df_colums_deleted.shape[1]}')
df_colums_deleted.head(2)

Quantidade de colunas antes da exclusão: 32
Quantidade de colunas após a exclusão: 26


,CONTRATACAO,NU_PRODUTO,NM_PRODUTO,REDE_ATENDIMENTO,DS_REDE_ATENDIMENTO,UF,CIDADE,TIPO_ESTABELECIMENTO,SERVICO,CD_ESPECIALIDADE,...,ENDERECO,COMPLEMENTO,NM_BAIRRO,CD_CEP,DS_FONE,SITE,NM_LIVRO,TIPO_PESSOA,IBGE,TIPO_PRESTADOR_SERVICO
0,2-Coletivo Empresarial,474363156,ADVANCE 600,SP103,ADVANCE 600,AC,RIO BRANCO,HOSPITAL ELETIVO,1,999,...,TV GETULIO VARGAS 1940,NaN,BOSQUE,69908650.0,6832241364/,NaN,AMICO LIMITADA,2,1200401,1-HOSPITAL
1,2-Coletivo Empresarial,474363156,ADVANCE 600,SP103,ADVANCE 600,AC,RIO BRANCO,SERVICOS AUXILIARES DE DIAGNOSTICO E TERAPIA,1,999,...,TV GETULIO VARGAS 1940,NaN,BOSQUE,69908650.0,6832241364/,NaN,AMICO LIMITADA,2,1200401,1-HOSPITAL


In [254]:
list_organized_columns = ['CONTRATACAO', 'NU_PRODUTO', 'NM_PRODUTO', 'REDE_ATENDIMENTO',
      'DS_REDE_ATENDIMENTO', 'UF', 'CIDADE', 'NM_PRESTADOR', 'NM_FANTASIA','TIPO_ESTABELECIMENTO',
      'SERVICO', 'DS_ESPECIALIDADE', 'CD_PRESTADOR','NU_CGC_CPF', 'NU_CNES_CRM',
      'ENDERECO','COMPLEMENTO', 'NM_BAIRRO', 'CD_CEP', 'DS_FONE', 'SITE', 'NM_LIVRO',
       'TIPO_PESSOA', 'IBGE', 'TIPO_PRESTADOR_SERVICO']

df_organized_columns = df_colums_deleted[list_organized_columns].copy()
df_organized_columns.head(2)

,CONTRATACAO,NU_PRODUTO,NM_PRODUTO,REDE_ATENDIMENTO,DS_REDE_ATENDIMENTO,UF,CIDADE,NM_PRESTADOR,NM_FANTASIA,TIPO_ESTABELECIMENTO,...,ENDERECO,COMPLEMENTO,NM_BAIRRO,CD_CEP,DS_FONE,SITE,NM_LIVRO,TIPO_PESSOA,IBGE,TIPO_PRESTADOR_SERVICO
0,2-Coletivo Empresarial,474363156,ADVANCE 600,SP103,ADVANCE 600,AC,RIO BRANCO,AMICO LIMITADA,PRONTO CLINICA,HOSPITAL ELETIVO,...,TV GETULIO VARGAS 1940,NaN,BOSQUE,69908650.0,6832241364/,NaN,AMICO LIMITADA,2,1200401,1-HOSPITAL
1,2-Coletivo Empresarial,474363156,ADVANCE 600,SP103,ADVANCE 600,AC,RIO BRANCO,AMICO LIMITADA,PRONTO CLINICA,SERVICOS AUXILIARES DE DIAGNOSTICO E TERAPIA,...,TV GETULIO VARGAS 1940,NaN,BOSQUE,69908650.0,6832241364/,NaN,AMICO LIMITADA,2,1200401,1-HOSPITAL


#### __4° Passo__: Modificando a coluna __`SERVICO`__.
- 1°: Troca os códigos abaixo para __`Consulta`__.
   - `0`
- 2°: Troca os códigos abaixo para __`Exame`__.
   - `19, 20, 21, 22, 23, 24`
   - `26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36`
   - `201, 202`
   - `401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412`
- 3°: Troca os códigos abaixo para __`Terapias`__.
   - `25`
   - `60, 61, 62, 63`
   - `500`
- 4°: Excluir todos o outros __`Códigos`__ que sobraram.

In [255]:
# vamos adcionar um dicionário e atráves dele vamos mapear os tipos de serviços
dict_map ={
   'CONSULTA': [0],
   'EXAME': [19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 201, 202, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412],
   'TERAPIAS': [25, 60, 61, 62, 63, 500],
}
print(dict_map.keys())
print(dict_map.values())

dict_keys(['CONSULTA', 'EXAME', 'TERAPIAS'])
dict_values([[0], [19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 201, 202, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412], [25, 60, 61, 62, 63, 500]])


In [256]:
df_organized_columns.SERVICO.unique()

array([  1,   0,  19,  23, 403,  28,  12, 201,  25,  39,  42, 500,  20,
        45,  31, 409, 411,  34,  36,  43,  21,  27, 413,  33,  32, 406,
       408, 307,  52,  80,  16,  29,  51, 414, 316, 202, 315, 401, 415,
       303,  35,  50,  54, 313, 410, 405, 412,  24,  17, 402, 301, 304,
        15,  26,  48,  60, 404,  61, 980,  22,  30,  41,  55, 310,  40,
        56,  49,  10, 407,  62, 312, 302,  47, 311, 314, 309, 306, 305,
        53,  44, 308,  46,  63,  65,  67,  57,  13], dtype=int64)

In [257]:
# agora vamos criar uma função para mapear os serviços
def map_services(row):
   #quando os valores que estão no dicionário forem iguais ao valor da coluna, então retorna a chave
   for key, value in dict_map.items():
      if row in value:
         return key
   return 'OUTROS'

# aplicando a função
df_organized_columns['SERVICO'] = df_organized_columns.SERVICO.apply(map_services)
df_organized_columns.head(2)

,CONTRATACAO,NU_PRODUTO,NM_PRODUTO,REDE_ATENDIMENTO,DS_REDE_ATENDIMENTO,UF,CIDADE,NM_PRESTADOR,NM_FANTASIA,TIPO_ESTABELECIMENTO,...,ENDERECO,COMPLEMENTO,NM_BAIRRO,CD_CEP,DS_FONE,SITE,NM_LIVRO,TIPO_PESSOA,IBGE,TIPO_PRESTADOR_SERVICO
0,2-Coletivo Empresarial,474363156,ADVANCE 600,SP103,ADVANCE 600,AC,RIO BRANCO,AMICO LIMITADA,PRONTO CLINICA,HOSPITAL ELETIVO,...,TV GETULIO VARGAS 1940,NaN,BOSQUE,69908650.0,6832241364/,NaN,AMICO LIMITADA,2,1200401,1-HOSPITAL
1,2-Coletivo Empresarial,474363156,ADVANCE 600,SP103,ADVANCE 600,AC,RIO BRANCO,AMICO LIMITADA,PRONTO CLINICA,SERVICOS AUXILIARES DE DIAGNOSTICO E TERAPIA,...,TV GETULIO VARGAS 1940,NaN,BOSQUE,69908650.0,6832241364/,NaN,AMICO LIMITADA,2,1200401,1-HOSPITAL


In [258]:
print(df_organized_columns.SERVICO.nunique())
print(df_organized_columns.SERVICO.unique())
print(df_organized_columns.SERVICO.value_counts())
# excluindo os servicos que são iguais a 'OUTROS'
df_services = df_organized_columns[df_organized_columns.SERVICO  !='OUTROS'].copy()
df_services.reset_index(drop=True, inplace=True)

print(f'Quantidade de linhas do df original: {df_organized_columns.shape}')
print(f'Quantidade de linhas filtradas: {df_services.shape}')
print(df_services.SERVICO.value_counts())


4
['OUTROS' 'CONSULTA' 'EXAME' 'TERAPIAS']
SERVICO
CONSULTA    57605
EXAME       53475
OUTROS      22385
TERAPIAS     8403
Name: count, dtype: int64
Quantidade de linhas do df original: (141868, 25)
Quantidade de linhas filtradas: (119483, 25)
SERVICO
CONSULTA    57605
EXAME       53475
TERAPIAS     8403
Name: count, dtype: int64


#### __5° Passo__: Filtros nas colunas `TIPO_PESSOA` e `TIPO_PRESTADOR_SERVICO`.
   - `TIPO_PESSOA` - Exluir filtrar o valores igaus __`1`__.
      - Segundo nosso amigo Colombiano, excluí somente onde tem Corpo clínico.
   - `TIPO_PRESTADOR_SERVICO` - Excluir todos as linhas que forem iguas a __`CORPO CLÍNILO`__.

In [259]:
import re

list_bady_clinique = [
   'CORPO CLINICLO', 'CORPO CLÍNICO', 'CORPO_CLINICO', 'CORPO_CLÍNICO',
   'Corpo Clinico', 'Corpo Clínico', 'Corpo_Clinico', 'Corpo_Clínico',
   'corpo clinico', 'corpo clínico', 'corpo_clinico', 'corpo_clínico',
]

# precisamos criar uma função para verificar se a string está contida na lista
def check_bad_clinique(row):
    pattern = '|'.join(list_bady_clinique)
    if re.search(pattern, row):
        return 'SIM'
    return 'NAO'

# aplicando a função
df_services['STATUS_CLINIQUE'] = df_services.TIPO_PRESTADOR_SERVICO.apply(check_bad_clinique)
print(df_services.STATUS_CLINIQUE.value_counts())
df_status_clinique = df_services[df_services.STATUS_CLINIQUE == 'NAO'].copy()

list_status_clinique = ['CONTRATACAO', 'NU_PRODUTO', 'NM_PRODUTO', 'REDE_ATENDIMENTO',
       'DS_REDE_ATENDIMENTO', 'UF', 'CIDADE', 'NM_PRESTADOR', 'NM_FANTASIA',
       'TIPO_ESTABELECIMENTO', 'SERVICO', 'DS_ESPECIALIDADE', 'CD_PRESTADOR',
       'NU_CGC_CPF', 'NU_CNES_CRM', 'ENDERECO', 'COMPLEMENTO', 'NM_BAIRRO',
       'CD_CEP', 'DS_FONE', 'SITE', 'NM_LIVRO', 'TIPO_PESSOA', 'IBGE',
       'TIPO_PRESTADOR_SERVICO']

df_status_clinique = df_status_clinique[list_status_clinique].copy()
df_status_clinique.head(2)

STATUS_CLINIQUE
NAO    119483
Name: count, dtype: int64


,CONTRATACAO,NU_PRODUTO,NM_PRODUTO,REDE_ATENDIMENTO,DS_REDE_ATENDIMENTO,UF,CIDADE,NM_PRESTADOR,NM_FANTASIA,TIPO_ESTABELECIMENTO,...,ENDERECO,COMPLEMENTO,NM_BAIRRO,CD_CEP,DS_FONE,SITE,NM_LIVRO,TIPO_PESSOA,IBGE,TIPO_PRESTADOR_SERVICO
0,2-Coletivo Empresarial,474363156,ADVANCE 600,SP103,ADVANCE 600 (SP103),AL,SAO MIGUEL DOS CAMPOS,SANTA CASA MISERIC S MIGUEL DOS CAMPOS,SANTA CASA MISERIC S MIGUEL DO,PRONTO SOCORRO URGEN EMERGEN,...,PC DR. JOSE INACIO 43,NaN,CENTRO,57240000.0,8232711389/,NaN,SANTA CASA MISERIC S MIGUEL DOS CAMPOS,2,2708600,1-HOSPITAL
1,2-Coletivo Empresarial,474363156,ADVANCE 600,SP103,ADVANCE 600 (SP103),AL,SAO MIGUEL DOS CAMPOS,SANTA CASA MISERIC S MIGUEL DOS CAMPOS,SANTA CASA MISERIC S MIGUEL DO,SERVICOS AUXILIARES DE DIAGNOSTICO E TERAPIA,...,PC DR. JOSE INACIO 43,NaN,CENTRO,57240000.0,8232711389/,NaN,SANTA CASA MISERIC S MIGUEL DOS CAMPOS,2,2708600,1-HOSPITAL


#### __6° Passo__: Classificação por determinadas colunas.
   -  __1 : `UF`__
   - __2 : `CIDADE`__
   - __3 : `NM_LIVRO`__
   - __4 : `ESPECIALIDADE`__

In [260]:
# Classificando o DataFrame pelas colunas desejadas
df_status_clinique_sorted = df_status_clinique.sort_values(by=['UF', 'CIDADE', 'NM_LIVRO', 'DS_ESPECIALIDADE'])

# Resetando o índice
df_status_clinique_sorted.reset_index(drop=True, inplace=True)

# Exibindo as primeiras linhas do DataFrame resultante
print(df_status_clinique_sorted.shape)
df_status_clinique_sorted.head(3)

(119483, 25)


,CONTRATACAO,NU_PRODUTO,NM_PRODUTO,REDE_ATENDIMENTO,DS_REDE_ATENDIMENTO,UF,CIDADE,NM_PRESTADOR,NM_FANTASIA,TIPO_ESTABELECIMENTO,...,ENDERECO,COMPLEMENTO,NM_BAIRRO,CD_CEP,DS_FONE,SITE,NM_LIVRO,TIPO_PESSOA,IBGE,TIPO_PRESTADOR_SERVICO
0,2-Coletivo Empresarial,474363156,ADVANCE 600,SP103,ADVANCE 600 (SP103),AL,ARAPIRACA,CLINICA PROCRIANCA S S LTDA,CLINICA PROCRIANCA S S LTDA,SERVICOS AUXILIARES DE DIAGNOSTICO E TERAPIA,...,R NOSSA SENHORA DE FATIMA 159,NaN,BRASILIA,57313040.0,8235309442/,NaN,CLINICA PROCRIANCA S S LTDA,2,2700300,1-HOSPITAL
1,2-Coletivo Empresarial,474363156,ADVANCE 600,SP103,ADVANCE 600 (SP103),AL,ARAPIRACA,CLINICA PROCRIANCA S S LTDA,CLINICA PROCRIANCA S S LTDA,SERVICOS AUXILIARES DE DIAGNOSTICO E TERAPIA,...,R SENHORA DE FATIMA 159,NaN,CENTRO,57300040.0,8235309442/,NaN,CLINICA PROCRIANCA S S LTDA,2,2700300,1-HOSPITAL
2,2-Coletivo Empresarial,474363156,ADVANCE 600,SP103,ADVANCE 600 (SP103),AL,ARAPIRACA,CLINICA PROCRIANCA S S LTDA,CLINICA PROCRIANCA S S LTDA,HOSPITAL ELETIVO,...,R NOSSA SENHORA DE FATIMA 159,NaN,BRASILIA,57313040.0,8235309442/,NaN,CLINICA PROCRIANCA S S LTDA,2,2700300,1-HOSPITAL


In [261]:
# Convertendo a coluna NU_CGC_CPF para string
df_status_clinique_sorted['NU_CGC_CPF'] = df_status_clinique_sorted['NU_CGC_CPF'].astype(str)
df_status_clinique_sorted['NU_CGC_CPF'] = df_status_clinique_sorted['NU_CGC_CPF'].str.replace('.0', '', regex=False)

# Apagando as informações que constam em NU_CGC_CPF onde TIPO_PESSOA é '1'
df_status_clinique_sorted.loc[df_status_clinique_sorted['TIPO_PESSOA'] == 1, 'NU_CGC_CPF'] = ''

# Excluindo a coluna CD_PRESTADOR
df_status_clinique_sorted.drop(columns=['CD_PRESTADOR'], inplace=True)

# Exibindo as primeiras linhas do DataFrame resultante
df_status_clinique_sorted.head(3)

,CONTRATACAO,NU_PRODUTO,NM_PRODUTO,REDE_ATENDIMENTO,DS_REDE_ATENDIMENTO,UF,CIDADE,NM_PRESTADOR,NM_FANTASIA,TIPO_ESTABELECIMENTO,...,ENDERECO,COMPLEMENTO,NM_BAIRRO,CD_CEP,DS_FONE,SITE,NM_LIVRO,TIPO_PESSOA,IBGE,TIPO_PRESTADOR_SERVICO
0,2-Coletivo Empresarial,474363156,ADVANCE 600,SP103,ADVANCE 600 (SP103),AL,ARAPIRACA,CLINICA PROCRIANCA S S LTDA,CLINICA PROCRIANCA S S LTDA,SERVICOS AUXILIARES DE DIAGNOSTICO E TERAPIA,...,R NOSSA SENHORA DE FATIMA 159,NaN,BRASILIA,57313040.0,8235309442/,NaN,CLINICA PROCRIANCA S S LTDA,2,2700300,1-HOSPITAL
1,2-Coletivo Empresarial,474363156,ADVANCE 600,SP103,ADVANCE 600 (SP103),AL,ARAPIRACA,CLINICA PROCRIANCA S S LTDA,CLINICA PROCRIANCA S S LTDA,SERVICOS AUXILIARES DE DIAGNOSTICO E TERAPIA,...,R SENHORA DE FATIMA 159,NaN,CENTRO,57300040.0,8235309442/,NaN,CLINICA PROCRIANCA S S LTDA,2,2700300,1-HOSPITAL
2,2-Coletivo Empresarial,474363156,ADVANCE 600,SP103,ADVANCE 600 (SP103),AL,ARAPIRACA,CLINICA PROCRIANCA S S LTDA,CLINICA PROCRIANCA S S LTDA,HOSPITAL ELETIVO,...,R NOSSA SENHORA DE FATIMA 159,NaN,BRASILIA,57313040.0,8235309442/,NaN,CLINICA PROCRIANCA S S LTDA,2,2700300,1-HOSPITAL


In [263]:
print(df_status_clinique_sorted[df_status_clinique_sorted.TIPO_PESSOA == 1]['NU_CGC_CPF'].unique())
print(df_status_clinique_sorted.shape)

['']
(119483, 24)


In [264]:
df_status_clinique_sorted.to_excel('df_status_clinique_sorted.xlsx', index=False)

In [5]:
import pandas as pd

df = pd.read_csv(r'modelo_de_busca.csv', sep=';', encoding='latin1')

print(df.shape)
df.head(10)

(6, 1)


,NU_PRODUTO
0,474363156
1,161181515
2,6518916515
3,51615121515
4,516516151
5,3550305 NDI


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   NU_PRODUTO  6 non-null      object
dtypes: object(1)
memory usage: 180.0+ bytes
